In [1]:
import pickle

In [2]:
import sys

In [3]:
import matgl

In [4]:
from matgl.ext.ase import PESCalculator, MolecularDynamics, Relaxer

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_std", torch.tensor(da

In [6]:
relaxer = Relaxer(pot)

In [7]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)
    

In [8]:
sys.getsizeof(structure_DB)

32636176

In [9]:
a = structure_DB.loc[0,'init_structure'].copy()

In [10]:
b = relaxer.relax(a)

In [11]:
from pymatgen.io import ase as pg_ase

ase_to_pmg = pg_ase.AseAtomsAdaptor.get_structure
pmg_to_ase = pg_ase.AseAtomsAdaptor.get_atoms

In [12]:
pmg_to_ase(b['final_structure'])

MSONAtoms(symbols='Al', pbc=True, cell=[[2.0371313531185136, 2.037133279011304, -4.723380494606584e-07], [1.267526692030542e-06, 2.0371315391501987, 2.0371283519273007], [2.0371317340363584, -8.532583613103346e-07, 2.0371304727132413]], calculator=PESCalculator(...))

In [13]:
b['trajectory'].energies[-1]

-3.7008004188537598

In [14]:
import random
import numpy as np

In [15]:
def run_relaxer(structure, rel):
    b = rel.relax(structure.copy())
    return b['trajectory'].atoms, b['trajectory'].energies[-1]

In [23]:
def relax_row(row, calculator_label, relaxer):
    return run_relaxer(row['init_structure'], relaxer)

In [29]:


def run_packet(df_global, calculator_label, relaxer, size_of_packet, structure_size):
    df = df_global#.copy()
    """ """
    
    if not calculator_label in df.columns:
        df[calculator_label] = None
        df[calculator_label] = df[calculator_label].astype(object)
        uncalculated = df.index
        
    else:
        uncalculated = df.attrs['uncalculated']
    #uncalculated_size = df[(df['size'] == structure_size) & uncalculated].index.to_list()
    uncalculated_size = list(
        set(df[df['size'] == structure_size].index.to_list()) & 
        set(uncalculated)) 
    
    rest_number = len(uncalculated_size)
    if rest_number > size_of_packet:
        to_calculate = random.sample(uncalculated_size, size_of_packet)
    else:
        to_calculate = uncalculated_size
        
    print(to_calculate)
    df.loc[to_calculate, calculator_label] = df.loc[to_calculate].apply(
        relax_row,args = (calculator_label, relaxer), axis=1)

    df.attrs['uncalculated'] = list(set(uncalculated) - set(to_calculate))

In [30]:
run_packet(structure_DB, 'm3g_test', relaxer, 2, 3)

[291, 108]


In [31]:
len(structure_DB.attrs['uncalculated'])

509932